Build dataset


In [1]:
from typing import TypedDict, Type, Any, Callable

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from spark.config import views
from spark.create_session import create_session

from IPython.display import display

from fitter import Fitter, get_common_distributions

In [2]:
VIEWS = views("v3")
spark = create_session()

for view, file in VIEWS.items():
    df = spark.read.json(file)
    df.createOrReplaceTempView(view)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/25 23:20:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/25 23:20:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
sessions_full = spark.sql(f"SELECT * FROM sessions ").toPandas()
sessions = spark.sql(f"SELECT DISTINCT user_id, track_id FROM sessions WHERE event_type like 'like'").toPandas()
tracks =  spark.sql(f"SELECT DISTINCT id, acousticness, danceability, duration_ms, energy, instrumentalness, key, liveness, loudness, popularity, EXTRACT(year from `release_date`) as release_year, speechiness, tempo, valence FROM tracks ").toPandas()

In [4]:
display(sessions_full)
display(sessions)
display(tracks)

,event_type,session_id,timestamp,track_id,user_id
0,play,124,2023-09-20T07:18:17,4flBNxpPLGVAdufOzRBIen,101
1,like,124,2023-09-20T07:18:30.069000,4flBNxpPLGVAdufOzRBIen,101
2,play,125,2023-04-27T00:20:57.181000,3uMYq07Kj5m564OQwdSCrD,101
3,play,126,2023-02-11T05:34:54.160000,2RChe0r2cMoyOvuKobZy44,101
4,advertisment,126,2023-02-11T05:34:56.685000,None,101
...,...,...,...,...,...
580241,like,102015,2023-11-12T09:06:01.022000,4cLdpErILMO8Db8pQVAVcZ,1100
580242,play,102015,2023-11-12T09:09:17.564000,3j8ja2Hq824OaRqIENJPTH,1100
580243,like,102015,2023-11-12T09:12:19.739000,3j8ja2Hq824OaRqIENJPTH,1100
580244,skip,102015,2023-11-12T09:12:58.093000,3j8ja2Hq824OaRqIENJPTH,1100


,user_id,track_id
0,101,4YKAOXyqkKZ5gOZR0AmVMN
1,104,5l9cr0lt9adgsQlhRAvgeL
2,105,2d8JP84HNLKhmd6IYOoupQ
3,108,4qO03RMQm88DdpTJcxlglY
4,113,5kcvBnt6DPX3AMEsCx1qDh
...,...,...
80366,1077,4GF00uRxC4ECrxNEbnapqf
80367,1090,4PMdq7Q7xOFrY424ZelZEb
80368,1100,4Dio1t8RHzLv1nfPJTSZyP
80369,1076,3omvXShuRPM3zbDpWYqf5g


,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,popularity,release_year,speechiness,tempo,valence
0,3KfbEIOC7YIv90FIfNSZpo,0.4490,0.688,146333,0.435,0.000000,9,0.1130,-11.359,74,1965,0.0323,103.239,0.435
1,1dxbAIfCASqv6jix2R1Taj,0.7840,0.698,148413,0.293,0.010600,0,0.0936,-11.361,55,1967,0.0332,117.613,0.724
2,17PXXzOygMyXXUNLngVN5u,0.8820,0.539,173733,0.114,0.009020,2,0.1060,-20.575,60,1970,0.0346,141.378,0.608
3,3Um9toULmYFGCpvaIPFw7l,0.4470,0.283,233000,0.716,0.000000,1,0.3990,-9.632,72,1971,0.0986,201.960,0.828
4,0GjEhVFGZW8afUYGChu3Rr,0.3580,0.543,230400,0.870,0.000939,9,0.7920,-6.514,81,1976,0.0428,100.804,0.754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22407,3VbsXTdxBl6IavGjFZbl3R,0.0394,0.720,225012,0.824,0.000000,10,0.0509,-5.086,53,2020,0.1280,165.998,0.333
22408,2zurBuX8TGOHkZLhye9Xme,0.0551,0.769,179080,0.922,0.000000,9,0.1940,-2.779,51,2020,0.1020,119.991,0.804
22409,3ev1ilsHdVbpXeT7FG6Kw7,0.6410,0.838,229187,0.461,0.000000,1,0.0825,-9.610,54,2017,0.2170,104.979,0.497
22410,5qTwnr6WbWeeUDS9dcl5g6,0.3370,0.444,303097,0.523,0.000000,1,0.1130,-4.522,54,2019,0.0286,185.872,0.420


In [5]:
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit(sessions['user_id'], sessions['track_id'])

# spark cannot handle this! lol
# item_features = spark.sql(f"SELECT DISTINCT s.user_id, s.track_id, acousticness FROM sessions s inner join tracks t WHERE event_type like 'like'").toPandas()


num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

(interactions, weights) = dataset.build_interactions(sessions[['user_id', 'track_id']].apply(tuple, axis=1))

print(repr(interactions))

item_features = dataset.build_item_features(zip(tracks['id'], tracks.drop('id', axis=1)))
print(repr(item_features))


Num users: 1000, num_items 21792.
<1000x21792 sparse matrix of type '<class 'numpy.int32'>'
	with 80371 stored elements in COOrdinate format>


ValueError: Feature a not in feature mapping. Call fit first.

In [ ]:
from lightfm import LightFM

model = LightFM(
    loss='warp',
    item_alpha=1e-6,
    user_alpha=1e-6,
    no_components=30
)
model.fit(
    interactions=interactions,
    item_features=,
    epochs=50, 
    num_threads=12)

In [ ]:
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k, reciprocal_rank
from lightfm.cross_validation import random_train_test_split

(train, test) = random_train_test_split(interactions)

train_auc = auc_score(model, train, num_threads=12).mean()
test_auc = auc_score(model, test, train_interactions=train, num_threads=12).mean()

train_precision = precision_at_k(model, train, k=10, num_threads=12).mean()
test_precision = precision_at_k(model, test, k=10, num_threads=12).mean()

train_recall = recall_at_k(model, train, k=10, num_threads=12).mean()
test_recall = recall_at_k(model, test, k=10, num_threads=12).mean()

train_reciprocal_rank = reciprocal_rank(model, train, num_threads=12).mean()
test_reciprocal_rank = reciprocal_rank(model, test, num_threads=12).mean()

print('AUC: train %.6f, test %.6f.' % (train_auc, test_auc))
print('Precision: train %.6f, test %.6f.' % (train_precision, test_precision))
print('Recall: train %.6f, test %.6f.' % (train_recall, test_recall))
print('Reciprocal rank: train %.6f, test %.6f.' % (train_reciprocal_rank, test_reciprocal_rank))
